今回は、feedforward neural networkを使って、固定ウィンドウ幅(左右３)の情報を使って単語の品詞タグを予測するモデルを学習してみます．  
かなりシンプルですが、高速に予測できるという魅力から、より強力なモデルが沢山存在する今でも、NLPのいろんなツールで使われてます．  
easyccg: https://github.com/mikelewis0/easyccg  
syntaxnet: https://github.com/tensorflow/models/tree/master/research/syntaxnet  
stanford parser: https://nlp.stanford.edu/software/lex-parser.shtml  
(イメージ)　　
<img src='images/ff.png'>
(画像: https://xbt.net/blog/what-is-enigma/)

POSタグ(Part-Of-Speech, 品詞タグ)のリスト:  
- ADJ: adjective
- ADP: adposition
- ADV: adverb
- AUX: auxiliary
- CCONJ: coordinating conjunction
- DET: determiner
- INTJ: interjection
- NOUN: noun
- NUM: numeral
- PART: particle
- PRON: pronoun
- PROPN: proper noun
- PUNCT: punctuation
- SCONJ: subordinating conjunction
- SYM: symbol
- VERB: verb
- X: other

In [1]:
# つかうライブラリの読み込み

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Input, Reshape
from keras.optimizers import SGD
from collections import Counter

# 👇無視してOK

/home/cl/masashi-y/.pyenv_elm13/versions/anaconda3-4.2.0/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

In [3]:
# NLPでよく出てくるCoNLLフォーマット
!head -20 data/test.conll

1	No	no	ADV	ADV	_	7	discourse	wsj_2300.1	_
2	,	,	PUNCT	PUNCT	_	7	punct	_	_
3	it	it	PRON	PRON	_	7	nsubj	_	_
4	was	be	VERB	VERB	_	7	cop	_	_
5	n't	not	PART	PART	_	7	neg	_	_
6	Black	Black	PROPN	PROPN	_	7	compound	_	_
7	Monday	Monday	PROPN	PROPN	_	0	root	_	_
8	.	.	PUNCT	PUNCT	_	7	punct	_	_

1	Once	once	ADV	ADV	_	2	advmod	wsj_2300.10	_
2	again	again	ADV	ADV	_	9	advmod	_	_
3	-LCB-	-lcb-	PUNCT	PUNCT	_	9	punct	_	_
4	the	the	DET	DET	_	5	det	_	_
5	specialists	specialist	NOUN	NOUN	_	9	nsubj	_	_
6	-RCB-	-rcb-	PUNCT	PUNCT	_	9	punct	_	_
7	were	be	VERB	VERB	_	9	cop	_	_
8	not	not	PART	PART	_	9	neg	_	_
9	able	able	ADJ	ADJ	_	24	ccomp	_	_
10	to	to	PART	PART	_	11	mark	_	_
11	handle	handle	VERB	VERB	_	9	xcomp	_	_


In [4]:
# CoNLLフォーマットを読み込む関数
def read_conll(file):
    res = []
    words = []
    tags = []
    for line in open(file):
        line = line.strip()
         # 空行ならそれまでにつくった文を出力
        if len(line) == 0:
            res.append((words, tags))
            words = []
            tags = []
        # 単語とタグを取り出す
        else:
            items = line.split('\t')
            words.append(items[1].lower()) # 小文字にしておく
            tags.append(items[3])
    return res

In [5]:
# 学習データ (training data)
train_sents = read_conll('data/train.conll')
# 評価用データ (test data)
test_sents = read_conll('data/test.conll')
# 開発データ (development data)
dev_sents = read_conll('data/dev.conll')

In [6]:
words, tags = test_sents[0] # 一番最初の文
print(words)
print(tags)

['no', ',', 'it', 'was', "n't", 'black', 'monday', '.']
['ADV', 'PUNCT', 'PRON', 'VERB', 'PART', 'PROPN', 'PROPN', 'PUNCT']


In [7]:
def sliding_windows(lst):
    res = []
    for i in range(len(lst) - 6):
        res.append(lst[i:i+7])
    return res

In [8]:
sliding_windows(list(range(10)))

[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 7],
 [2, 3, 4, 5, 6, 7, 8],
 [3, 4, 5, 6, 7, 8, 9]]

In [9]:
print(words)
print(sliding_windows(words))

['no', ',', 'it', 'was', "n't", 'black', 'monday', '.']
[['no', ',', 'it', 'was', "n't", 'black', 'monday'], [',', 'it', 'was', "n't", 'black', 'monday', '.']]


👆これはだめ

In [10]:
PAD = 'PAD'

train_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in train_sents]
test_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in test_sents]
dev_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in dev_sents]

In [11]:
words, tags = test_sents[0]
print(words)
print(sliding_windows(words))

['PAD', 'PAD', 'PAD', 'no', ',', 'it', 'was', "n't", 'black', 'monday', '.', 'PAD', 'PAD', 'PAD']
[['PAD', 'PAD', 'PAD', 'no', ',', 'it', 'was'], ['PAD', 'PAD', 'no', ',', 'it', 'was', "n't"], ['PAD', 'no', ',', 'it', 'was', "n't", 'black'], ['no', ',', 'it', 'was', "n't", 'black', 'monday'], [',', 'it', 'was', "n't", 'black', 'monday', '.'], ['it', 'was', "n't", 'black', 'monday', '.', 'PAD'], ['was', "n't", 'black', 'monday', '.', 'PAD', 'PAD'], ["n't", 'black', 'monday', '.', 'PAD', 'PAD', 'PAD']]


In [12]:
#単語を自然数のIDに変換する辞書
UNK = 'UNK'

# 単語の埋め込みベクトルがうまくいくためには、その単語がいろいろな文脈で出現してほしい．
# 学習データにちょっと(２回より下)しか出ない単語はUNKで置き換える．
word_count = Counter(word for words, _ in train_sents for word in words)
word_set = [word for word, count in word_count.most_common() if count >= 2]
word_set.append(UNK)
word_dict = {w: i for i, w in enumerate(word_set)}

In [13]:
#POSタグを自然数のIDに変換する辞書
tag_set = set(tag for _, tags in train_sents for tag in tags)
tag_dict = {w: i for i, w in enumerate(tag_set)}

In [14]:
print('word_dict size', len(word_dict))
print('tag_dict size', len(tag_dict))

word_dict size 21569
tag_dict size 17


In [15]:
print(word_dict['dog']) # dogのidは？

6640


In [16]:
# データを行列に変換
xs = []
ys = []
for words, tags in train_sents:
    for window in sliding_windows(words):
        xs.append([word_dict.get(word, word_dict[UNK]) for word in window])
    ys.extend(tag_dict[tag] for tag in tags)

In [17]:
# numpyの行列に変換
xs = np.array(xs, 'i')
ys = np.array(ys, 'i')
ys = keras.utils.to_categorical(ys)

In [18]:
print('dimensions of xs', xs.shape)
print('dimensions of ys', ys.shape)

dimensions of xs (929552, 7)
dimensions of ys (929552, 17)


In [19]:
# test, devも行列にしたいので関数にする
def make_matrices(words_and_tags):
    xs = []
    ys = []
    for words, tags in words_and_tags:
        for window in sliding_windows(words):
            xs.append([word_dict.get(word, word_dict[UNK]) for word in window])
        ys.extend(tag_dict[tag] for tag in tags)

    xs = np.array(xs, 'i')
    ys = np.array(ys, 'i')
    ys = keras.utils.to_categorical(ys, len(tag_dict))
    print('dimensions of xs', xs.shape)
    print('dimensions of ys', ys.shape)
    return xs, ys

In [20]:
train_xs, train_ys = make_matrices(train_sents)
test_xs, test_ys = make_matrices(test_sents)
dev_xs, dev_ys = make_matrices(dev_sents)

dimensions of xs (929552, 7)
dimensions of ys (929552, 17)
dimensions of xs (55371, 7)
dimensions of ys (55371, 17)
dimensions of xs (45422, 7)
dimensions of ys (45422, 17)


In [21]:
VOCAB_SIZE = len(word_dict)  # 単語数
EMBED_DIM = 128                      # 埋め込みベクトルの次元数
HIDDEN1_DIM = 256                   # 隠れ層１
HIDDEN2_DIM = 128                   # 隠れ層２
NUM_TAGS = len(tag_dict)

model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_DIM))
model.add(Reshape((EMBED_DIM * 7,)))
model.add(Dense(HIDDEN1_DIM, activation='tanh'))
model.add(Dense(HIDDEN2_DIM, activation='tanh'))
model.add(Dense(NUM_TAGS, activation='softmax'))

In [22]:
keras.utils.plot_model(model, 'images/model.png')

単語のID列${\bf x} = x_{-2}, x_{-1}, x, x_{+1}, x_{+2}$に対して  
$Embedding(\bf x) = [ {\bf e}_{x_{-2}}　| {\bf e}_{x_{-1}}　| {\bf e}_{x}　| {\bf e}_{x_{+1}}　|　{\bf e}_{x_{+2}} ]^T　= {\bf E}^T$,  
$ Reshape({\bf E}) = [ {\bf e}_{x_{-2}}, {\bf e}_{x_{-1}}, {\bf e}_{x}, {\bf e}_{x_{+1}}, {\bf e}_{x_{+2}} ]^T = {\bf e}$ (縦に並べる),  
$f({\bf x}) = {\mathit softmax}(W_3 \tanh (W_2 \tanh (W_1 {\bf e} + b_1) + b_2) + b_3)$.

計算グラフの可視化
<img src='images/model.png'>

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2760832   
_________________________________________________________________
reshape_1 (Reshape)          (None, 896)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               229632    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 17)                2193      
Total params: 3,025,553
Trainable params: 3,025,553
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [25]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4787645517783878182, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7082170778
 locality {
   bus_id: 1
 }
 incarnation: 18102384936197633674
 physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0, compute capability: 5.2", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6375404340
 locality {
   bus_id: 1
 }
 incarnation: 9913575533825859606
 physical_device_desc: "device: 1, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2"]

In [26]:
# 学習
model.fit(train_xs, train_ys, batch_size=1024, epochs=200, verbose=1, validation_data=(dev_xs, dev_ys))

# 学習がめんどい場合こっち (学習済みのパラメータを読み込み)
# model.load_weights('models/weights.h5')

Train on 929552 samples, validate on 45422 samples
Epoch 1/200
929552/929552 [==============================] - 12s 13us/step - loss: 2.4425 - acc: 0.2286 - val_loss: 2.1993 - val_acc: 0.3461
Epoch 2/200
929552/929552 [==============================] - 4s 4us/step - loss: 1.9336 - acc: 0.4257 - val_loss: 1.7043 - val_acc: 0.4768
Epoch 3/200
929552/929552 [==============================] - 6s 6us/step - loss: 1.6029 - acc: 0.4992 - val_loss: 1.4710 - val_acc: 0.5298
Epoch 4/200
929552/929552 [==============================] - 5s 5us/step - loss: 1.3961 - acc: 0.5549 - val_loss: 1.2861 - val_acc: 0.5893
Epoch 5/200
929552/929552 [==============================] - 5s 5us/step - loss: 1.2266 - acc: 0.6119 - val_loss: 1.1398 - val_acc: 0.6354
Epoch 6/200
929552/929552 [==============================] - 6s 7us/step - loss: 1.0822 - acc: 0.6569 - val_loss: 1.0063 - val_acc: 0.6788
Epoch 7/200
929552/929552 [==============================] - 6s 6us/step - loss: 0.9473 - acc: 0.6995 - val_loss:

929552/929552 [==============================] - 5s 5us/step - loss: 0.1552 - acc: 0.9498 - val_loss: 0.2062 - val_acc: 0.9339
Epoch 60/200
929552/929552 [==============================] - 5s 5us/step - loss: 0.1536 - acc: 0.9502 - val_loss: 0.2053 - val_acc: 0.9343
Epoch 61/200
929552/929552 [==============================] - 5s 6us/step - loss: 0.1520 - acc: 0.9508 - val_loss: 0.2043 - val_acc: 0.9348
Epoch 62/200
929552/929552 [==============================] - 5s 5us/step - loss: 0.1505 - acc: 0.9513 - val_loss: 0.2031 - val_acc: 0.9352
Epoch 63/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1490 - acc: 0.9518 - val_loss: 0.2017 - val_acc: 0.9355
Epoch 64/200
929552/929552 [==============================] - 7s 7us/step - loss: 0.1475 - acc: 0.9523 - val_loss: 0.2009 - val_acc: 0.9359
Epoch 65/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.1461 - acc: 0.9528 - val_loss: 0.2004 - val_acc: 0.9362
Epoch 66/200
929552/929552 [=====

929552/929552 [==============================] - 5s 6us/step - loss: 0.1047 - acc: 0.9661 - val_loss: 0.1812 - val_acc: 0.9436
Epoch 118/200
929552/929552 [==============================] - 5s 6us/step - loss: 0.1042 - acc: 0.9662 - val_loss: 0.1804 - val_acc: 0.9444
Epoch 119/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.1037 - acc: 0.9662 - val_loss: 0.1809 - val_acc: 0.9444
Epoch 120/200
929552/929552 [==============================] - 7s 8us/step - loss: 0.1033 - acc: 0.9665 - val_loss: 0.1808 - val_acc: 0.9444
Epoch 121/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.1028 - acc: 0.9666 - val_loss: 0.1807 - val_acc: 0.9443
Epoch 122/200
929552/929552 [==============================] - 5s 6us/step - loss: 0.1024 - acc: 0.9668 - val_loss: 0.1805 - val_acc: 0.9442
Epoch 123/200
929552/929552 [==============================] - 5s 5us/step - loss: 0.1019 - acc: 0.9668 - val_loss: 0.1800 - val_acc: 0.9443
Epoch 124/200
929552/929552

929552/929552 [==============================] - 7s 8us/step - loss: 0.0840 - acc: 0.9725 - val_loss: 0.1812 - val_acc: 0.9455
Epoch 176/200
929552/929552 [==============================] - 3s 4us/step - loss: 0.0837 - acc: 0.9725 - val_loss: 0.1812 - val_acc: 0.9457
Epoch 177/200
929552/929552 [==============================] - 6s 6us/step - loss: 0.0834 - acc: 0.9727 - val_loss: 0.1815 - val_acc: 0.9459
Epoch 178/200
929552/929552 [==============================] - 5s 5us/step - loss: 0.0831 - acc: 0.9727 - val_loss: 0.1818 - val_acc: 0.9456
Epoch 179/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.0828 - acc: 0.9729 - val_loss: 0.1810 - val_acc: 0.9456
Epoch 180/200
929552/929552 [==============================] - 8s 8us/step - loss: 0.0826 - acc: 0.9731 - val_loss: 0.1820 - val_acc: 0.9458
Epoch 181/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.0823 - acc: 0.9730 - val_loss: 0.1813 - val_acc: 0.9460
Epoch 182/200
929552/929552

In [27]:
# 学習したモデルを適当に使ってみるとなんか行列が出てきます
model.predict(test_xs[:10])

array([[8.10557082e-02, 1.32073769e-02, 7.10472639e-04, 7.18352795e-01,
        5.89033652e-06, 3.89746914e-04, 5.57578518e-04, 2.58904400e-08,
        1.58037595e-03, 2.01247330e-03, 1.02006830e-03, 7.10095046e-04,
        4.66211577e-06, 2.16930785e-04, 3.60750452e-09, 1.80175677e-01,
        7.04472061e-16],
       [7.86494547e-06, 2.18097651e-09, 1.24326913e-08, 6.10316864e-13,
        5.12558381e-06, 3.25297265e-07, 2.90912703e-05, 5.02471835e-07,
        8.15867799e-16, 5.47013144e-07, 9.99931097e-01, 5.55866075e-10,
        3.77816776e-14, 2.23222092e-12, 2.88937202e-14, 2.54073948e-05,
        2.39162648e-17],
       [5.88764760e-06, 9.99991417e-01, 3.21845732e-08, 2.37876435e-07,
        4.31857744e-11, 3.83840426e-09, 3.75526753e-07, 3.62298209e-07,
        8.90485342e-12, 1.44225919e-06, 1.40812717e-09, 4.49098270e-09,
        4.32071739e-12, 1.78185413e-08, 1.58697949e-16, 3.02612193e-07,
        8.71168284e-21],
       [2.78762332e-06, 2.99380929e-07, 6.22993355e-08, 1.895

In [29]:
# その行列の形
Out[27].shape

(10, 17)

In [30]:
# POSタグとIDの逆向きの辞書
rev_tag_dict = {v: k for k, v in tag_dict.items()}

In [32]:
# なにかでてきました
[rev_tag_dict[i] for i in np.argmax(Out[27], 1)]

['DET', 'PUNCT', 'PRON', 'VERB', 'PART', 'ADJ', 'PROPN', 'PUNCT', 'ADV', 'ADV']

In [35]:
# 単語リストを入力してPOSタグを予測する関数
def predict(words):
    words = [PAD] * 3 + words + [PAD] * 3
    ids = [word_dict.get(word, word_dict[UNK]) for word in words]
    windows = sliding_windows(ids)
    matrix = np.array(windows, 'i')
    probabilities = model.predict(matrix)
    result_ids = np.argmax(probabilities, 1)
    result = [rev_tag_dict[i] for i in result_ids]
    return result

In [36]:
predict(['this', 'is', 'a', 'test', 'sentence', '.'])

['PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'PUNCT']

In [43]:
import random
for _ in range(5):
    i = random.randint(0, len(test_sents))
    words, tags = test_sents[i]
    words = words[3:-3]
    print('sentence:\t', ' '.join(words))
    print('predict:\t', ' '.join(predict(words)))
    print('answer:\t', ' '.join(tags))

sentence:	 the president , at a news conference friday , also renewed a call for the ouster of panama 's noriega .
predict:	 DET NOUN PUNCT ADP DET NOUN NOUN PROPN PUNCT ADV VERB DET NOUN ADP DET NOUN ADP PROPN PART PROPN PUNCT
answer:	 DET NOUN PUNCT ADP DET NOUN NOUN PROPN PUNCT ADV VERB DET NOUN ADP DET NOUN ADP PROPN PART PROPN PUNCT
sentence:	 also , the company 's hair-growing drug , rogaine , is selling well -- at about $ 125 million for the year , but the company 's profit from the drug has been reduced by upjohn 's expensive print and television campaigns for advertising , analysts said .
predict:	 ADV PUNCT DET NOUN PART ADJ NOUN PUNCT ADJ PUNCT AUX VERB ADV PUNCT ADP ADP SYM NUM NUM ADP DET NOUN PUNCT CONJ DET NOUN PART NOUN ADP DET NOUN AUX AUX VERB ADP PROPN PART ADJ NOUN CONJ NOUN NOUN ADP NOUN PUNCT NOUN VERB PUNCT
answer:	 ADV PUNCT DET NOUN PART ADJ NOUN PUNCT PROPN PUNCT AUX VERB ADV PUNCT ADP ADV SYM NUM NUM ADP DET NOUN PUNCT CONJ DET NOUN PART NOUN ADP DET NOUN AUX

In [39]:
model.save_weights('models/weights.h5')